In [1]:
import torch
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
import torch.nn as nn
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(MLPModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 10)
        )
    
    def forward(self, input):
        input = input.view(input.size(0), -1)
        return self.layers(input)

In [3]:
class PR_CNN(nn.Module):
    def __init__(self, **kwargs):
        super(PR_CNN, self).__init__()
        self.expected_input_size = kwargs.get('input_dim', None)

        # First layer
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=kwargs.get('input_channels', None),
                      out_channels=kwargs.get('input_channels', None)*5,
                      kernel_size=3,
                      stride=3),
            nn.LeakyReLU()
        )
        # Second layer
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=kwargs.get('input_channels', None)*5,
                      out_channels=kwargs.get('input_channels', None)*25,
                      kernel_size=3,
                      stride=3),
            nn.LeakyReLU()
        )
        # Third layer
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=kwargs.get('input_channels', None)*25,
                      out_channels=1536,
                      kernel_size=3,
                      stride=3),
            nn.LeakyReLU()
        )
        

        # Classification layer
        self.fc = nn.Sequential(
            Flatten(),
            nn.Linear(1536, kwargs.get('output_channels', None))
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc(x)
        return x

In [4]:
import numpy as np
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def train(model, train_loader, optimizer, loss_fn, print_every=100):
    '''
    Trains the model for one epoch
    '''
    model.train()
    losses = []
    n_correct = 0
    for iteration, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        optimizer.zero_grad()
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
#         if iteration % print_every == 0:
#             print('Training iteration {}: loss {:.4f}'.format(iteration, loss.item()))
        losses.append(loss.item())
        n_correct += torch.sum(output.argmax(1) == labels).item()
    accuracy = 100.0 * n_correct / len(train_loader.dataset)
    return np.mean(np.array(losses)), accuracy
            
def test(model, test_loader, loss_fn):
    '''
    Tests the model on data from test_loader
    '''
    model.eval()
    test_loss = 0
    n_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()
            n_correct += torch.sum(output.argmax(1) == labels).item()

    average_loss = test_loss / len(test_loader)
    accuracy = 100.0 * n_correct / len(test_loader.dataset)
#     print('Test average loss: {:.4f}, accuracy: {:.3f}'.format(average_loss, accuracy))
    return average_loss, accuracy


def fit(train_dataloader, val_dataloader, model, optimizer, loss_fn, n_epochs, scheduler=None):
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    for epoch in range(n_epochs):
        train_loss, train_accuracy = train(model, train_dataloader, optimizer, loss_fn)
        val_loss, val_accuracy = test(model, val_dataloader, loss_fn)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if scheduler:
            scheduler.step() # argument only needed for ReduceLROnPlateau
        print('Epoch {}/{}: train_loss: {:.4f}, train_accuracy: {:.4f}, val_loss: {:.4f}, val_accuracy: {:.4f}'.format(epoch+1, n_epochs,
                                                                                                          train_losses[-1],
                                                                                                          train_accuracies[-1],
                                                                                                          val_losses[-1],
                                                                                                          val_accuracies[-1]))
    
    return val_accuracies

In [5]:
transform_dataset = transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])

In [6]:
train_dataset = datasets.ImageFolder('mnist/train',transform=transform_dataset)
test_dataset = datasets.ImageFolder('mnist/test',transform=transform_dataset)

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=100, shuffle=True)

In [11]:
train_perm_dataset = datasets.ImageFolder('mnist-permutated/train',transform=transform_dataset)
test_perm_dataset = datasets.ImageFolder('mnist-permutated/test',transform=transform_dataset)

In [12]:
train_perm_loader = torch.utils.data.DataLoader(train_perm_dataset,batch_size=100, shuffle=True)
test_perm_loader = torch.utils.data.DataLoader(test_perm_dataset,batch_size=100, shuffle=True)

In [14]:
nb_epochs = 25
hidden_layers = 512
learning_rate = 0.001
model_mlp = MLPModel(3*28*28, hidden_layers)
model_mlp = model_mlp.to(device)
optimizer = torch.optim.Adam(model_mlp.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
accuracies = fit(train_loader, test_loader, model_mlp, optimizer, loss_fn, nb_epochs)
accuracies_perm = fit(train_perm_loader, test_perm_loader, model_mlp, optimizer, loss_fn, nb_epochs)
print("Accuracy of the MLP with the normal dataset: ", accuracies[-1])
print("Accuracy of the MLP with the permuted dataset: ", accuracies_perm[-1])       

KeyboardInterrupt: 

In [ ]:
# Remplaces par tes valeurs + change le nom du model
nb_epochs = 25
learning_rate = 0.001
pre_defined_kwargs = {'input_dim': (28,28), 'input_channels': 3, 'output_channels': 10}
model_cnn = PR_CNN(**pre_defined_kwargs)
model_cnn = model_cnn.to(device)
optimizer = torch.optim.Adam(model_cnn.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
accuracies = fit(train_loader, test_loader, model_cnn, optimizer, loss_fn, nb_epochs)
accuracies_perm = fit(train_perm_loader, test_perm_loader, model_cnn, optimizer, loss_fn, nb_epochs)
print("Accuracy of the CNN with the normal dataset: ", accuracies[-1])
print("Accuracy of the CNN with the permuted dataset: ", accuracies_perm[-1])   